3.1 Optimization problem for given situation

\begin{align}
min.\sum_{i=1}^{6} |b_{0} + b_{1}*L_{i}+b_{2}*E_{i}-P_{i}|  \nonumber 
\end{align}
Here 'i' represents Warehouse no.  
$L_{i}$ is the Lot Size for warehouse 'i'  
$E_{i}$ is the Elevation for warehouse 'i'  
$P_{i}$ is the Selling Prize for warehouse 'i'

The quantity $b_{0} + b_{1}*L_{i}+b_{2}*E_{i}$ is the predicted price using the linear for warehouse 'i' and hence $|b_{0} + b_{1}*L_{i}+b_{2}*E_{i}-P_{i}|$ is the absolute value of error between actual price and predicted price.

We need to find our values of $b_{0},b_{1},b_{2}$ such that the sum of absolute value of errors is minimized.

$b_{0}$>=0  
$b_{1}$ & $b_{2}$ can take any real values.

This optimization problem is not linear because it includes absolute values and absolute value function is not a linear function.

In [44]:
!pip install -q pyomo
from pyomo.environ import * 
import numpy as np
import pandas as pd
!apt-get install -y -qq coinor-cbc

**3.5 Solving by approach 1**

We define variables     
$y_{i}$=$|b_{0} + b_{1}*L_{i}+b_{2}*E_{i}-P_{i}|$   
for 'i' in {1,2,3,4,5,6}

So objective is   
\begin{align}
min.\sum_{i=1}^{6} y_{i}  \nonumber 
\end{align}

s.t.  
$y_{i}>=0$  
$y_{i}-$ ($b_{0} + b_{1}*L_{i}+b_{2}*E_{i}-P_{i}$)$>=0$  
$y_{i}+$ ($b_{0} + b_{1}*L_{i}+b_{2}*E_{i}-P_{i}$)$>=0$

In [45]:
#3.5 by approach 1

data_csvfile = pd.read_csv('lab6_ex3.csv')

model_1 = ConcreteModel()

M=len(data_csvfile.columns)-1
N=len(data_csvfile.index)

col_indices = range(N)
row_indices = range(M)


#defining variables
model_1.y = Var(col_indices, domain=NonNegativeReals)
model_1.b = Var(row_indices)

model_1.b[0].domain=NonNegativeReals

#defining objective
model_1.objective = Objective(expr=model_1.y[0]+model_1.y[1]+model_1.y[2]+model_1.y[3]+model_1.y[4]+model_1.y[5] ,sense=minimize)

# adding constraints
model_1.constraints = ConstraintList()

for i in col_indices:
  model_1.constraints.add(expr=model_1.y[i]+model_1.b[0]+model_1.b[1]*data_csvfile['LotSize'][i]+model_1.b[2]*data_csvfile['Elevation'][i]-data_csvfile['SellingPrice'][i]>=0)
  model_1.constraints.add(expr=model_1.y[i]-model_1.b[0]-model_1.b[1]*data_csvfile['LotSize'][i]-model_1.b[2]*data_csvfile['Elevation'][i]+data_csvfile['SellingPrice'][i]>=0)

model_1.pprint()

3 Set Declarations
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

2 Var Declarations
    b : Size=3, Index=b_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :  None :  None :  None : False :  True :            Reals
          2 :  None :  None :  None : False :  True :            Reals
    y : Size=6, Index=y_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          

In [46]:
opt_cbc = SolverFactory('cbc')
result = opt_cbc.solve(model_1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [47]:
print('\noptimal sum of residuals = ', model_1.objective())

print("\n")
for j in row_indices:
  print("b[",j,"]=",model_1.b[j].value)

print("\n")

for i in col_indices:
  print("y[",i,"]=",model_1.y[i].value)



optimal sum of residuals =  125454.545


b[ 0 ]= 104181.82
b[ 1 ]= 67.272727
b[ 2 ]= -356.36364


y[ 0 ]= 38818.182
y[ 1 ]= 0.0
y[ 2 ]= 54272.727
y[ 3 ]= 32363.636
y[ 4 ]= 0.0
y[ 5 ]= 0.0


**3.5 Solving by approach 2**

We define varibles $u_{i}$ & $v_{i}$ for 'i' in {1,2,3,4,5,6}

s.t.  
$u_{i}>=0$  
$v_{i}>=0$     
$u_{i}-v_{i}=b_{0} + b_{1}*L_{i}+b_{2}*E_{i}-P_{i}$ 

Objective. 
\begin{align}
min.\sum_{i=1}^{6} u_{i}+v_{i}  \nonumber 
\end{align}

In [48]:
# 3.5 by approach 2

data_csvfile = pd.read_csv('lab6_ex3.csv')

model_2 = ConcreteModel()

M=len(data_csvfile.columns)-1
N=len(data_csvfile.index)

col_indices = range(N)
row_indices = range(M)

#defining variables
model_2.u = Var(col_indices, domain=NonNegativeReals)
model_2.v = Var(col_indices, domain=NonNegativeReals)

model_2.b = Var(row_indices)

model_2.b[0].domain=NonNegativeReals

#defining objective
model_2.objective = Objective(expr=model_2.u[0]+model_2.u[1]+model_2.u[2]+model_2.u[3]+model_2.u[4]+model_2.u[5]+model_2.v[0]+model_2.v[1]+model_2.v[2]+model_2.v[3]+model_2.v[4]+model_2.v[5],sense=minimize)

# adding constraints
model_2.constraints = ConstraintList()

for i in col_indices:
  model_2.constraints.add(expr=model_2.u[i]-model_2.v[i]-model_2.b[0]-model_2.b[1]*data_csvfile['LotSize'][i]-model_2.b[2]*data_csvfile['Elevation'][i]+data_csvfile['SellingPrice'][i]==0)

model_2.pprint()



4 Set Declarations
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    u_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    v_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

3 Var Declarations
    b : Size=3, Index=b_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :  None :  None :  None : False :  True :            Reals
          2 :  None :  None :  None : False :  True :            Reals
    u : Size=6, Index=u_index
        Ke

In [49]:
opt_cbc = SolverFactory('cbc')
result = opt_cbc.solve(model_2)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [50]:
print('\nObjective = ', model_2.objective())
print("\n")
for j in row_indices:
  print("b[",j,"]=",model_2.b[j].value)

print("\n")

for i in col_indices:
  print("u[",i,"]=",model_2.u[i].value)
  print("v[",i,"]=",model_2.v[i].value)



Objective =  125454.545


b[ 0 ]= 104181.82
b[ 1 ]= 67.272727
b[ 2 ]= -356.36364


u[ 0 ]= 0.0
v[ 0 ]= 38818.182
u[ 1 ]= 0.0
v[ 1 ]= 0.0
u[ 2 ]= 54272.727
v[ 2 ]= 0.0
u[ 3 ]= 32363.636
v[ 3 ]= 0.0
u[ 4 ]= 0.0
v[ 4 ]= 0.0
u[ 5 ]= 0.0
v[ 5 ]= 0.0


We can see that the value of decision variables at optimum and sum of residuals is exactly same by both the approaches.

Also note that we get value of $b_{1}$ as positive which indicates that as the lot size increases the price of warehouse increases. However the value of $b_{2}$ is negative which indicates that as the elevation increases the price of warehouse goes down.